In [ ]:
import cv2 as cv    
import numpy as np
#import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import os
from skimage import io, img_as_float, img_as_ubyte, exposure
from os.path import isfile, join
import os.path, sys

import glob


import re

In [ ]:
from ipywidgets import interact

%matplotlib notebook

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split(r'(\d+)', text) ]

def autoscale(array, percentile):
    value = np.percentile(np.ndarray.flatten(array), percentile)
    return value

## Step 1: Rename
This step is not essential, I just want to rename it numerically to make the name shorter. 

In [ ]:
path='/Users/rubyjiang/Desktop/TI64/Speed0.6(10:8)/70_Ti_Plate_t1.0mm_p70_0.6mps_S2F-2.5mm_U18G14_Ar_S1'
img_directory= path + '/'

In [ ]:
# Create a folder
processed0 = img_directory + 'processed0/'

try:
    os.makedirs(processed0)
except FileExistsError:
    pass

In [ ]:
# Rename the images and save
imgList=sorted(glob.glob(img_directory + '*.tif'))
i = 1
for image in imgList:   
    img = cv.imread(image, cv.IMREAD_GRAYSCALE)
    if i<10:
        name = 'frame00' + str(i) + '.tif'
    elif i<100: 
        name = 'frame0' + str(i) + '.tif'
    else:
        name = 'frame' + str(i) + '.tif'
    i +=1   
    cv.imwrite(processed0 + '/' + name, img)

## Step 2: image processing

### 2_1: Tao trick or JJ trick

In [ ]:
# Read processed0 images to list DXR_images
DXR_images = []

for file in sorted(glob.glob(processed0 + '/*.tif'), key=natural_keys):
    image = cv.imread(file, cv.IMREAD_UNCHANGED)
    DXR_images.append(image)
    
DXR_images = np.array(DXR_images)

In [ ]:
# Option 1 Tao trick
#Tao_images = DXR_images[1:] / DXR_images[0]

In [ ]:
# Option 2 JJ trick
JJ_images = []

jj_num = 30 #50 
DXR_images_hi = DXR_images[jj_num:DXR_images.shape[0]]
DXR_images_lo = DXR_images[0:(DXR_images.shape[0]-jj_num)]

index = 0
while index < (DXR_images.shape[0]-jj_num):
    for img in DXR_images_hi:
        hi = DXR_images_hi[index] / DXR_images_lo[index]# remove the black band
        lo = DXR_images_lo[index] / DXR_images_hi[index]
        JJ_images.append(hi)
        index += 1
JJ_images = np.array(JJ_images)

In [ ]:
# Display
show_images = JJ_images

# # min and max pixel intensities for auto scaling for plotting img
pixel_min1 = 0   
pixel_max1 = 100
######################################################################################
fig, ax = plt.subplots(dpi=100)
im = plt.imshow(show_images[0], cmap='gray')
#fig.colorbar(im)
# ax.set_xlabel('X (pixels)')
plt.xticks([])
# ax.set_ylabel('Y (pixels)')
plt.yticks([])
loop_num = np.arange(len(show_images))
@interact(frame_num = (loop_num[0], loop_num[-1]))
def show(frame_num):
    im.set_array(show_images[frame_num])
    im.set_clim(autoscale(show_images[frame_num], pixel_min1), autoscale(show_images[frame_num], pixel_max1))
    fig.canvas.draw_idle()
    fig.canvas.draw_idle()

### 2_2: clip, normalize, equa

In [ ]:
JJ_images[250].max()

In [ ]:
clip_images = []
norm_images = []
equa_images = []


for jj in JJ_images:
    #1
    clip = np.clip(jj, np.percentile(jj, 1), np.percentile(jj, 99))
    #2
    norm = cv.normalize(clip, None, alpha=0, beta=255, norm_type=cv.NORM_MINMAX, dtype = cv.CV_8U)
#   #3_1
#   equa = rank.equalize(norm, selem=disk(30))
    '''
    This example enhances an image with low contrast, using a method called 
    local histogram equalization, 
    which spreads out the most frequent intensity values in an image.
    '''
    #3_2
    # create a CLAHE object (Arguments are optional).
    clahe = cv.createCLAHE(clipLimit=2.0, tileGridSize=(2,2))
    cl1 = clahe.apply(norm)

    # Append
    clip_images.append(clip)
    norm_images.append(norm)
    equa_images.append(cl1)

### 2_3: Blur and threshold 

In [ ]:
blur_images = []
for jj in equa_images:
    img = cv.medianBlur(jj,5)
    blur_images.append(img)

### Plot the histogram of the above three 

In [ ]:
#Visulize the images

import matplotlib
from skimage.util.dtype import dtype_range

matplotlib.rcParams['font.size'] = 9

def plot_img_and_hist(image, axes, bins=256):
    """Plot an image along with its histogram and cumulative histogram.

    """
    ax_img, ax_hist = axes
    ax_cdf = ax_hist.twinx()

    # Display image
    ax_img.imshow(image, cmap=plt.cm.gray)
    ax_img.set_axis_off()

    # Display histogram
    ax_hist.hist(image.ravel(), bins=bins)
    ax_hist.ticklabel_format(axis='y', style='scientific', scilimits=(0, 0))
    ax_hist.set_xlabel('Pixel intensity')

    xmin, xmax = dtype_range[image.dtype.type]
    ax_hist.set_xlim(xmin, xmax)

    # Display cumulative distribution
    img_cdf, bins = exposure.cumulative_distribution(image, bins)
    ax_cdf.plot(bins, img_cdf, 'r')
    
    return ax_img, ax_hist, ax_cdf

# numb =250
# # # clip 
# # img = clip_images[num ]

# # norm
# img_n = norm_images[numb ] 

# # equa
# img_e = equa_images[numb ]

# # blur
# img_b = blur_images[numb ]

# # Display results
# fig = plt.figure(figsize=(8, 5))
# axes = np.zeros((2, 3), dtype=np.object)
# axes[0, 0] = plt.subplot(2, 3, 1)
# axes[0, 1] = plt.subplot(2, 3, 2, sharex=axes[0, 0], sharey=axes[0, 0])
# axes[0, 2] = plt.subplot(2, 3, 3, sharex=axes[0, 0], sharey=axes[0, 0])
# axes[1, 0] = plt.subplot(2, 3, 4)
# axes[1, 1] = plt.subplot(2, 3, 5)
# axes[1, 2] = plt.subplot(2, 3, 6)

# ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_n, axes[:, 0])
# ax_img.set_title('norm')
# ax_hist.set_ylabel('Number of pixels')

# ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_e, axes[:, 1])
# ax_img.set_title('equa')

# ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_b, axes[:, 2])
# ax_img.set_title('blur')
# ax_cdf.set_ylabel('Fraction of total intensity')


# # prevent overlap of y-axis labels
# fig.tight_layout()

In [ ]:
jj=blur_images[250]
np.percentile(jj, 93)

In [ ]:
thre_images = []
for jj in blur_images:
    retval, thres  = cv.threshold(jj,
                                   np.percentile(jj, 93), 
                                   # about 225, any pixel above this value is assigned as white, below as black
                                   255,
                                   cv.THRESH_BINARY)
    thre_images.append(thres)

In [ ]:
# Create a folder to save the thre_images
folder = img_directory + 'threshold/'
try:
    os.makedirs(folder)
except FileExistsError:
    pass
################################################
i = jj_num + 1 
for img in thre_images:   
    #img = cv.imread(image, cv.IMREAD_GRAYSCALE)
    if i<10:
        name = 'frame00' + str(i) + '.tif'
    elif i<100: 
        name = 'frame0' + str(i) + '.tif'
    else:
        name = 'frame' + str(i) + '.tif'
    i +=1   
    cv.imwrite(folder + '/' + name, img)

### 2_4: Polyfill to have clean images

In [ ]:
clean_images = []

for img in thre_images:
    #img = cv.imread(frame,0)
    contours,hierarchy = cv.findContours(img, 1, 2)
    areas = [cv.contourArea(contour) for contour in contours]
    # if your segmented clean images has noises, this step will not work properly. 
    # you need to make sure that your segmented mask only have one chunk of white pixels, i.e., the keyhole
    if len(areas)==0:
        clean_images.append(clean_images)
    else:
        keyhole_index = np.argmax(areas)
        # use fillpoly to make pores black
        pores_index = [i for i in range(len(areas)) if i != keyhole_index]
        clean_image = cv.fillPoly(img,pts= [contours[i] for i in pores_index],color = 0)
        clean_images.append(clean_image)

### 2_5: close the images 

In [ ]:
#kernel = np.ones((5,5),np.uint8)
kernel = np.ones((10,10),np.uint8)
close_images = []
for img in clean_images:
    closing = cv.morphologyEx(img, cv.MORPH_CLOSE, kernel)
    close_images.append(closing)

In [ ]:
# Create a folder to save the close_images
folder = img_directory + 'close/'
try:
    os.makedirs(folder)
except FileExistsError:
    pass
################################################
i = jj_num +1 # num is used in the jj trick
for img in close_images:   
    #img = cv.imread(image, cv.IMREAD_GRAYSCALE)
    if i<10:
        name = 'frame00' + str(i) + '.tif'
    elif i<100: 
        name = 'frame0' + str(i) + '.tif'
    else:
        name = 'frame' + str(i) + '.tif'
    i +=1   
    cv.imwrite(folder + '/' + name, img)

# Check the images!

In [ ]:
# Display
show_images =  thre_images # thre_images

# # min and max pixel intensities for auto scaling for plotting img
pixel_min1 = 0   
pixel_max1 = 255
######################################################################################
fig, ax = plt.subplots(dpi=100)
im = plt.imshow(show_images[0], cmap='gray')
#fig.colorbar(im)
# ax.set_xlabel('X (pixels)')
plt.xticks([])
# ax.set_ylabel('Y (pixels)')
plt.yticks([])
loop_num = np.arange(len(show_images))
@interact(frame_num = (loop_num[0], loop_num[-1]))
def show(frame_num):
    im.set_array(show_images[frame_num])
    im.set_clim(autoscale(show_images[frame_num], pixel_min1), autoscale(show_images[frame_num], pixel_max1))
    fig.canvas.draw_idle()

### If you only want to see 1 image

In [ ]:
# from scipy import ndimage, misc
# import matplotlib.pyplot as plt
# fig = plt.figure()
# plt.gray()  # show the filtered result in grayscale
# ax1 = fig.add_subplot(121)  # left side
# ax2 = fig.add_subplot(122)  # right side
# ascent = thre_images[250]
# result = ndimage.median_filter(ascent, size=5)
# ax1.imshow(ascent)
# ax2.imshow(result)
# plt.show()

# Plot jj, clip, norm, equa, blur, thre, clean, close (8)

In [ ]:
numb = 100#250

img_jj = JJ_images[numb]
img_clip = clip_images[numb]
img_norm = norm_images[numb] 
img_equa = equa_images[numb]

img_blur = blur_images[numb]
img_thre = thre_images[numb]
img_clea = clean_images[numb]
img_clos = close_images[numb]

# Display results
fig = plt.figure(figsize=(21, 5))
axes = np.zeros((2, 8), dtype=np.object)
axes[0, 0] = plt.subplot(2, 8, 1)
axes[0, 1] = plt.subplot(2, 8, 2, sharex=axes[0, 0], sharey=axes[0, 0])
axes[0, 2] = plt.subplot(2, 8, 3, sharex=axes[0, 0], sharey=axes[0, 0])
axes[0, 3] = plt.subplot(2, 8, 4, sharex=axes[0, 0], sharey=axes[0, 0])
axes[0, 4] = plt.subplot(2, 8, 5, sharex=axes[0, 0], sharey=axes[0, 0])
axes[0, 5] = plt.subplot(2, 8, 6, sharex=axes[0, 0], sharey=axes[0, 0])
axes[0, 6] = plt.subplot(2, 8, 7, sharex=axes[0, 0], sharey=axes[0, 0])
axes[0, 7] = plt.subplot(2, 8, 8, sharex=axes[0, 0], sharey=axes[0, 0])

axes[1, 0] = plt.subplot(2, 8, 9)
axes[1, 1] = plt.subplot(2, 8, 10)
axes[1, 2] = plt.subplot(2, 8, 11)
axes[1, 3] = plt.subplot(2, 8, 12)
axes[1, 4] = plt.subplot(2, 8, 13)
axes[1, 5] = plt.subplot(2, 8, 14)
axes[1, 6] = plt.subplot(2, 8, 15)
axes[1, 7] = plt.subplot(2, 8, 16)



ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_jj, axes[:, 0])
ax_img.set_title('JJ')
ax_hist.set_ylabel('Number of pixels')

ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_clip, axes[:, 1])
ax_img.set_title('clip')


ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_norm, axes[:, 2])
ax_img.set_title('norm')


ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_equa, axes[:, 3])
ax_img.set_title('equa')

ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_blur, axes[:, 4])
ax_img.set_title('blur')


ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_thre, axes[:, 5])
ax_img.set_title('threshold')


ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_clea, axes[:, 6])
ax_img.set_title('clean')


ax_img, ax_hist, ax_cdf = plot_img_and_hist(img_clos, axes[:, 7])
ax_img.set_title('close')
ax_cdf.set_ylabel('Fraction of total intensity')


# prevent overlap of y-axis labels
fig.tight_layout()

In [ ]:
#fig.savefig(img_directory+'8_analysis.jpg',dpi=300)